In [ ]:
import os
from PIL import Image
from glob import glob

# Clean Script
from glob import glob
import tensorflow as tf
from tensorflow.keras.utils import load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D

# Collect paths to images based on label
nonCancerPaths = glob('imagedata/*/0/*.png')
cancerousPaths = glob('imagedata/*/1/*.png')

# Turn filepaths into image arrays to train a model
def paths_to_image(paths, label):
    images = []
    labels = []
    for path in paths:
        image = load_img(path)
        image = image.resize([50, 50])
        imgArray = tf.keras.utils.img_to_array(image)
        images.append(imgArray)
        labels.append(label)
    return (images, labels)
nonCancerImages = paths_to_image(nonCancerPaths, 0)
cancerImages = paths_to_image(cancerousPaths, 1)

from sklearn.model_selection import train_test_split
allImages = nonCancerImages[0] + cancerImages[0]
allLabels = nonCancerImages[1] + cancerImages[1]

x_train, x_test, y_train, y_test = train_test_split(allImages, allLabels, test_size=.5, stratify= allLabels, random_state = 10)

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np


print(np.array(x_train).shape)
print(np.array(y_train).shape)

#Add data augmentation to increase the robustness of the model
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])


base_model = ResNet50(weights= None, include_top=False, input_shape= (50, 50,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

print("FITTING")
history = model.fit(np.array(x_train), np.array(y_train), epochs=100, validation_data=(np.array(x_test), np.array(y_test)), batch_size=64)



